# Example analysis of some included samples

In service of reproducibility of our results, we include the samples used to generate the figures in our papers.  We include some examples below of how one might produce similar plots given the data we provide.

## Preamble

In [ ]:
%load_ext autoreload
%autoreload
import numpy as np
import sys
import jax.numpy as jnp

sys.path.append("../..")
import linx.const as const
from linx.background import BackgroundModel
from linx.thermo import N_eff

from scipy.interpolate import interp1d

import matplotlib
import matplotlib.pyplot as plt
import corner
from matplotlib.lines import Line2D

%matplotlib inline

We sample the input parameter `dNeff`, which does not map on to $N_{\rm{eff}}$ in a straightforward way.  So we will build an interpolation we can use to go between our `dNeff` samples and our desired $N_{\rm{eff}}$ posterior:

In [ ]:
get_thermodynamics_Neff = BackgroundModel()
step = 0.1
input_dNeff = np.arange(-10.,10+step,step=step)
output_Neff = []
for dNeff in input_dNeff:
    t_vec, a_vec, rho_g_vec, rho_nu_vec, rho_extra_vec, P_extra_vec, Neff_vec = get_thermodynamics_Neff(jnp.asarray(dNeff))

    output_Neff.append(N_eff(rho_g_vec[-1] + 3*rho_nu_vec[-1] + rho_extra_vec[-1],rho_g_vec[-1]))
Neff = interp1d(input_dNeff,output_Neff)

Now we can call `Neff` on our samples where needed.  Let's also set up arrays of parameter names to make including labels easier:

In [ ]:
reactions = [
    'npdg', 'dpHe3g', 'ddHe3n', 'ddtp', 'tpag', 'tdan', 'taLi7g', 'He3ntp', 
    'He3dap', 'He3aBe7g', 'Be7nLi7p', 'Li7paa', 'Li7paag', 'Be7naa', 'Be7daap', 
    'daLi6g', 'Li6pBe7g', 'Li6pHe3a', 'B8naap', 'Li6He3aap', 'Li6taan', 
    'Li6tLi8p', 'Li7He3Li6a', 'Li8He3Li7a', 'Be7tLi6a', 'B8tBe7a'
]

CMB_nuisances = ['A_cib_217', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 
                 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 'gal545_A_100', 'gal545_A_143',
                 'gal545_A_143_217', 'gal545_A_217', 'galf_Te_A_100', 'galf_Te_A_100_143', 
                 'galf_Te_A_100_217', 'galf_Te_A_143', 'galf_Te_A_143_217', 'galf_Te_A_217', 
                 'calib_100T', 'calib_217T', 'A_planck']

## BBN only

Here's how we might view the results of sampling without BBN nuisance parameters ("no_marg" indicates BBN nuisance parameters were not marginalized over, apart from the neutron lifetime `tau_n_fac`):

In [ ]:
samples = np.load('../samples/BBN_only/dynesty_key_PRIMAT_2023_BBN_only_no_marg_omegab_samples.npz')
labelArr = ['omega_b','tau_n_fac']
fig = corner.corner(samples['samples'],labels=labelArr)

print("omega_b results")

median = np.median(samples['samples'][:,0])
lower_bound = np.percentile(samples['samples'][:,0], 16) # 50 +/- 34 to get 68% CL
upper_bound = np.percentile(samples['samples'][:,0], 84)
print("median: ",median)
print("lower sigma: ",median - lower_bound)
print("upper sigma: ",upper_bound - median)

And here's how we'd do it adding the BBN nuisance parameters.  Let's also include $N_{\rm{eff}}$ to see how transforming the samples works:

In [ ]:
samples = np.load('../samples/BBN_only/dynesty_key_NACREII_BBN_only_marg_omegab_Neff_samples.npz')
reconf_arr = np.concatenate((samples['samples'][:,0].reshape(-1,1),
                             Neff(samples['samples'][:,1]).reshape(-1,1),
                             samples['samples'][:,2:]),axis=1)

labelArr = np.concatenate(([r'$100 \omega_b h^2$',r'$N_{\rm{eff}}$',r'tau_n_fac'],reactions))
fig = corner.corner(reconf_arr,labels=labelArr,label_kwargs={"fontsize": 18})


print("omega_b")
median = np.median(samples['samples'][:,0])
lower_bound = np.percentile(samples['samples'][:,0], 16)
upper_bound = np.percentile(samples['samples'][:,0], 84)
print("median: ",median)
print("lower sigma: ",median - lower_bound)
print("upper sigma: ",upper_bound - median)

print("Neff")
median = np.median(Neff(samples['samples'][:,1]))
lower_bound = np.percentile(Neff(samples['samples'][:,1]), 16)
upper_bound = np.percentile(Neff(samples['samples'][:,1]), 84)
print("median: ",median)
print("lower sigma: ",median - lower_bound)
print("upper sigma: ",upper_bound - median)

## CMB only

Here's a quick example of CMB-only samples, including CMB nuisance parameters ("nuisance" indicates the CMB nuisance parameters were sampled):

In [ ]:
samples = np.load('../samples/CMB_only/dynesty_CMB_only_nuisance_omegab_Neff_samples.npz') 

labelArr = np.concatenate((['$A_s$','$n_s$','$h$',r'$100 \Omega_b h^2$',r'$\Omega_{\rm{CDM}}$',
                            r'$\tau_{\rm{reio}}$',r'$N_{\rm{eff}}$'],CMB_nuisances))
fig = corner.corner(samples['samples'],labels=labelArr)

print("omega_b")
median = np.median(samples['samples'][:,3])
lower_bound = np.percentile(samples['samples'][:,3], 16)
upper_bound = np.percentile(samples['samples'][:,3], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

print("Neff")
median = np.median(samples['samples'][:,6])
lower_bound = np.percentile(samples['samples'][:,6], 16)
upper_bound = np.percentile(samples['samples'][:,6], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

It's hard to see anything with all of those nuisance parameters, so let's just look at the model parameters:

In [ ]:
samples = np.load('../samples/CMB_only/dynesty_CMB_only_nuisance_omegab_Neff_samples.npz') 

labelArr = np.concatenate((['$A_s$','$n_s$','$h$',r'$100 \Omega_b h^2$',r'$\Omega_{\rm{CDM}}$',
                            r'$\tau_{\rm{reio}}$',r'$N_{\rm{eff}}$'],CMB_nuisances))
fig = corner.corner(samples['samples'][:,:7],labels=labelArr)

print("omega_b")
median = np.median(samples['samples'][:,3])
lower_bound = np.percentile(samples['samples'][:,3], 16)
upper_bound = np.percentile(samples['samples'][:,3], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

print("Neff")
median = np.median(samples['samples'][:,6])
lower_bound = np.percentile(samples['samples'][:,6], 16)
upper_bound = np.percentile(samples['samples'][:,6], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

## Joint analyses

We can try to just plot everything, but beware the output of this cell is large!

In [ ]:
samples = np.load('../samples/Joint_analyses/dynesty_key_PArth_CMB_BBN_marg_nuisance_omegab_Neff_samples.npz')
reconf_arr = np.concatenate((samples['samples'][:,:6],Neff(samples['samples'][:,6]).reshape(-1,1),samples['samples'][:,7:]),axis=1)

labelArr = np.concatenate((['$A_s$','$n_s$','$h$',r'$100 \Omega_b h^2$',r'$\Omega_{\rm{CDM}}$',
                            r'$\tau_{\rm{reio}}$',r'$N_{\rm{eff}}$','tau_n_fac'],CMB_nuisances,reactions))
fig = corner.corner(reconf_arr,labels=labelArr)


print("omega_b")
median = np.median(samples['samples'][:,3])
lower_bound = np.percentile(samples['samples'][:,3], 16)
upper_bound = np.percentile(samples['samples'][:,3], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

print("Neff")
median = np.median(Neff(samples['samples'][:,6]))
lower_bound = np.percentile(Neff(samples['samples'][:,6]), 16)
upper_bound = np.percentile(Neff(samples['samples'][:,6]), 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

Best instead to just plot the model parameters:

In [ ]:
samples = np.load('../samples/Joint_analyses/dynesty_key_PArth_CMB_BBN_marg_nuisance_omegab_Neff_samples.npz')
reconf_arr = np.concatenate((samples['samples'][:,:6],Neff(samples['samples'][:,6]).reshape(-1,1),samples['samples'][:,7:]),axis=1)

labelArr = np.concatenate((['$A_s$','$n_s$','$h$',r'$100 \Omega_b h^2$',r'$\Omega_{\rm{CDM}}$',
                            r'$\tau_{\rm{reio}}$',r'$N_{\rm{eff}}$','tau_n_fac'],CMB_nuisances,reactions))
fig = corner.corner(reconf_arr[:,:7],labels=labelArr)


print("omega_b")
median = np.median(samples['samples'][:,3])
lower_bound = np.percentile(samples['samples'][:,3], 16)
upper_bound = np.percentile(samples['samples'][:,3], 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)

print("Neff")
median = np.median(Neff(samples['samples'][:,6]))
lower_bound = np.percentile(Neff(samples['samples'][:,6]), 16)
upper_bound = np.percentile(Neff(samples['samples'][:,6]), 84)
print("median: ",median)
print("lower: ",median - lower_bound)
print("upper: ",upper_bound - median)